El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [20]:
# Manipulación y análisis de datos
import pandas as pd
import numpy as np
from datetime import datetime

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y Métrica
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Modelos
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor

Procedemos a leer los datos de la base de datos usando la librería pandas, una vez importados los datos, procederemos a ver los datos, en que condicion estan, si faltan valores y en que cantidad. Tambien si los tipos de datos corresponden a la columna.

In [2]:
df=pd.read_csv('car_data.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [3]:

print("Conteo de valores nulos por columna:")
missing_data = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100
missing_info = pd.DataFrame({'Missing Count': missing_data,'Percentage': missing_percentage}).sort_values('Percentage', ascending=False)
print(missing_info)




Conteo de valores nulos por columna:
                   Missing Count  Percentage
NotRepaired                71154   20.079070
VehicleType                37490   10.579368
FuelType                   32895    9.282697
Gearbox                    19833    5.596709
Model                      19705    5.560588
Price                          0    0.000000
RegistrationYear               0    0.000000
DateCrawled                    0    0.000000
Mileage                        0    0.000000
Power                          0    0.000000
RegistrationMonth              0    0.000000
Brand                          0    0.000000
DateCreated                    0    0.000000
NumberOfPictures               0    0.000000
PostalCode                     0    0.000000
LastSeen                       0    0.000000


In [4]:
df[['NotRepaired','VehicleType','FuelType','Gearbox','Model']] = df[['NotRepaired','VehicleType','FuelType','Gearbox','Model']].fillna('unknown')
df[['VehicleType','Gearbox','FuelType','NotRepaired']]=df[['VehicleType','Gearbox','FuelType','NotRepaired']].astype('category')


Voy a comenzar a tratar los valores ausentes. en el caso de "NotRepaired" como puede ser si o no, y tenemos este valor pero tiene un alto porcentaje de mi dataset, pondre otro valor como "unknow".

In [5]:
print("Conteo de valores nulos por columna:")
missing_data = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100
missing_info = pd.DataFrame({'Missing Count': missing_data,'Percentage': missing_percentage}).sort_values('Percentage', ascending=False)
print(missing_info)


Conteo de valores nulos por columna:
                   Missing Count  Percentage
DateCrawled                    0         0.0
Price                          0         0.0
VehicleType                    0         0.0
RegistrationYear               0         0.0
Gearbox                        0         0.0
Power                          0         0.0
Model                          0         0.0
Mileage                        0         0.0
RegistrationMonth              0         0.0
FuelType                       0         0.0
Brand                          0         0.0
NotRepaired                    0         0.0
DateCreated                    0         0.0
NumberOfPictures               0         0.0
PostalCode                     0         0.0
LastSeen                       0         0.0


Terminamos de ver el tema de valores ausentes , como las columnas eran categoricas por eso se procedio a imputar en todas la palabra "Unknown". Ahora procederemos a ver la coherencia de los datos.

In [6]:
print(df.describe())

               Price  RegistrationYear          Power        Mileage  \
count  354369.000000     354369.000000  354369.000000  354369.000000   
mean     4416.656776       2004.234448     110.094337  128211.172535   
std      4514.158514         90.227958     189.850405   37905.341530   
min         0.000000       1000.000000       0.000000    5000.000000   
25%      1050.000000       1999.000000      69.000000  125000.000000   
50%      2700.000000       2003.000000     105.000000  150000.000000   
75%      6400.000000       2008.000000     143.000000  150000.000000   
max     20000.000000       9999.000000   20000.000000  150000.000000   

       RegistrationMonth  NumberOfPictures     PostalCode  
count      354369.000000          354369.0  354369.000000  
mean            5.714645               0.0   50508.689087  
std             3.726421               0.0   25783.096248  
min             0.000000               0.0    1067.000000  
25%             3.000000               0.0   30165.

Observamos varias anomalais en los datos, como que en precio hhubo una venta por 0 euros, en el caso del año del registro hay desde 1000 hasta 9999, en el caso de power hay un valor de 20000 y en el kilometraje hay un valor de 9999999, todos estos valores no son coherentes con la realidad, por lo que se procederá a eliminarlos.

In [7]:
df['Price']=df['Price'].drop(df[df['Price']<=100].index)
df['Power']=df['Power'].drop(df[(df['Power']>=10000) | (df['Power']<10)].index)
df['RegistrationYear']=df['RegistrationYear'].drop(df[(df['RegistrationYear']<1900) | (df['RegistrationYear']>2025)].index)
print(df.describe())



               Price  RegistrationYear          Power        Mileage  \
count  340024.000000     354198.000000  313879.000000  354369.000000   
mean     4602.471902       2003.084789     122.330895  128211.172535   
std      4514.902742          7.536418     106.232815   37905.341530   
min       101.000000       1910.000000      10.000000    5000.000000   
25%      1200.000000       1999.000000      75.000000  125000.000000   
50%      2900.000000       2003.000000     110.000000  150000.000000   
75%      6500.000000       2008.000000     150.000000  150000.000000   
max     20000.000000       2019.000000    9710.000000  150000.000000   

       RegistrationMonth  NumberOfPictures     PostalCode  
count      354369.000000          354369.0  354369.000000  
mean            5.714645               0.0   50508.689087  
std             3.726421               0.0   25783.096248  
min             0.000000               0.0    1067.000000  
25%             3.000000               0.0   30165.

Listo tenemos el df limpio y listo para entrenar el modelo.

## Entrenamiento del modelo 

## Entrenamiento de modelo de Regresión lineal

In [8]:
# --- INICIA EL BLOQUE DE CÓDIGO MÁS ROBUSTO ---

# Carga tus datos (si no lo has hecho)
# df = pd.read_csv('tu_archivo.csv', encoding='ISO-8859-1') 

# 1. FORZAR LA COLUMNA 'Price' A SER NUMÉRICA (NUEVO PASO CRÍTICO)
# El parámetro errors='coerce' convertirá cualquier valor que no sea un número en NaN.
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# 2. AHORA SÍ, ELIMINAR TODAS LAS FILAS DONDE 'Price' SEA NULO
# Esto eliminará tanto los nulos originales como los que acabamos de crear en el paso anterior.
df.dropna(subset=['Price'], inplace=True)

# 3. Limpieza de columnas y anomalías (el resto del código que ya conoces)
df_cleaned = df.drop(columns=['DateCrawled', 'DateCreated', 'LastSeen', 'PostalCode', 'NumberOfPictures', 'RegistrationMonth'])
current_year = datetime.now().year
df_cleaned = df_cleaned[(df_cleaned['RegistrationYear'] >= 1950) & (df_cleaned['RegistrationYear'] <= current_year)]
df_cleaned = df_cleaned[(df_cleaned['Power'] >= 50) & (df_cleaned['Power'] <= 1000)]
df_cleaned['CarAge'] = current_year - df_cleaned['RegistrationYear']
df_cleaned = df_cleaned.drop(columns=['RegistrationYear'])

# 4. Rellenar nulos restantes en las CARACTERÍSTICAS (X)
numerical_cols = df_cleaned.select_dtypes(include=np.number).columns.drop('Price')
categorical_cols = df_cleaned.select_dtypes(exclude=np.number).columns

for col in numerical_cols:
    df_cleaned[col].fillna(df_cleaned[col].median(), inplace=True)

for col in categorical_cols:
    df_cleaned[col].fillna(df_cleaned[col].mode()[0], inplace=True)

# 5. Codificación y División de Datos
df_final = df_cleaned.copy()
categorical_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
for col in categorical_features:
    df_final[col] = df_final[col].astype('category')

df_ohe = pd.get_dummies(df_final, columns=categorical_features, drop_first=True)

X_cat = df_final.drop('Price', axis=1)
y = df_final['Price']
X_ohe = df_ohe.drop('Price', axis=1)

X_train_cat, X_test_cat, y_train, y_test = train_test_split(X_cat, y, test_size=0.2, random_state=42)
X_train_ohe, X_test_ohe, y_train_ohe, y_test_ohe = train_test_split(X_ohe, y, test_size=0.2, random_state=42)

print("¡Preparación de datos completada sin errores!")
print("Total de nulos en y_train_ohe:", y_train_ohe.isnull().sum())

# --- FIN DEL BLOQUE ---

C:\Users\oslo_\AppData\Local\Temp\ipykernel_23684\217730971.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned[col].fillna(df_cleaned[col].median(), inplace=True)
C:\Users\oslo_\AppData\Local\Temp\ipykernel_23684\217730971.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

¡Preparación de datos completada sin errores!
Total de nulos en y_train_ohe: 0


In [9]:
print(f"Tamaño del conjunto de entrenamiento (categorical): {X_train_cat.shape}")
print(f"Tamaño del conjunto de prueba (categorical): {X_test_cat.shape}")
print(f"Tamaño del conjunto de entrenamiento (ohe): {X_train_ohe.shape}")
print(f"Tamaño del conjunto de prueba (ohe): {X_test_ohe.shape}")

Tamaño del conjunto de entrenamiento (categorical): (239623, 9)
Tamaño del conjunto de prueba (categorical): (59906, 9)
Tamaño del conjunto de entrenamiento (ohe): (239623, 310)
Tamaño del conjunto de prueba (ohe): (59906, 310)


In [10]:
print(df_ohe.isnull().sum())
print(X_train_ohe.isnull().sum().sort_values(ascending=False))

Price                      0
Power                      0
Mileage                    0
CarAge                     0
VehicleType_convertible    0
                          ..
Brand_trabant              0
Brand_volkswagen           0
Brand_volvo                0
NotRepaired_unknown        0
NotRepaired_yes            0
Length: 311, dtype: int64
Power                      0
Mileage                    0
CarAge                     0
VehicleType_convertible    0
VehicleType_coupe          0
                          ..
Brand_trabant              0
Brand_volkswagen           0
Brand_volvo                0
NotRepaired_unknown        0
NotRepaired_yes            0
Length: 310, dtype: int64


## Análisis del modelo

## Modelo 1: Regresión Lineal

In [23]:
%%time

lr = LinearRegression()
lr.fit(X_train_ohe, y_train_ohe)

y_pred_linear = lr.predict(X_test_ohe)

rmse_linear = np.sqrt(mean_squared_error(y_test_ohe, y_pred_linear))
print(f"RMSE de Regresión Lineal: {rmse_linear:.2f} euros")

RMSE de Regresión Lineal: 2557.41 euros
CPU times: total: 30.2 s
Wall time: 5.97 s


## Modelo 2: Random Forest 

In [12]:
%%time
rf_model = RandomForestRegressor(n_estimators=100,max_depth=20, random_state=123, n_jobs=-1)

# 2. Entrenar el modelo con los datos OHE
print("Entrenando el modelo Random Forest...")
rf_model.fit(X_train_ohe, y_train_ohe)
print("Entrenamiento completado.")

# 3. Hacer predicciones y evaluar
y_pred_rf = rf_model.predict(X_test_ohe)
rmse_rf = np.sqrt(mean_squared_error(y_test_ohe, y_pred_rf))

print(f"\nRMSE de Random Forest: {rmse_rf:.2f} euros")

Entrenando el modelo Random Forest...
Entrenamiento completado.

RMSE de Random Forest: 1572.58 euros
CPU times: total: 22min 42s
Wall time: 2min 38s


## Modelo 3: Decision Tree

In [21]:
%%time
dt_model = DecisionTreeRegressor(max_depth=10, random_state=123)
print("Entrenando el modelo Random Forest...")
dt_model.fit(X_train_ohe, y_train_ohe)
print("Entrenamiento completado.")
y_pred_dt = dt_model.predict(X_test_ohe)
rmse_dt = np.sqrt(mean_squared_error(y_test_ohe, y_pred_dt))
print(f"RMSE de Decision Tree: {rmse_dt:.2f} euros")

Entrenando el modelo Random Forest...
Entrenamiento completado.
RMSE de Decision Tree: 1977.94 euros
CPU times: total: 4.5 s
Wall time: 4.56 s


## Modelo 4: LightGBM

In [24]:
%%time

lgbm_model = lgb.LGBMRegressor(random_state=42)
lgbm_model.fit(X_train_cat, y_train)

y_pred_lgbm = lgbm_model.predict(X_test_cat)
rmse_lgbm = np.sqrt(mean_squared_error(y_test, y_pred_lgbm))

print(f"RMSE de LightGBM (default): {rmse_lgbm:.2f} euros")



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009490 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 642
[LightGBM] [Info] Number of data points in the train set: 239623, number of used features: 9
[LightGBM] [Info] Start training from score 4897.793284
RMSE de LightGBM (default): 1608.73 euros
CPU times: total: 5.08 s
Wall time: 946 ms


## Modelo 5: CatBoost

In [25]:
%%time
cat_features_names = X_train_cat.select_dtypes(include=['category']).columns.tolist()

# Inicializar y entrenar el modelo
cat_model = CatBoostRegressor(random_state=42, verbose=0, cat_features=cat_features_names)
cat_model.fit(X_train_cat, y_train)
y_pred_cat = cat_model.predict(X_test_cat)
rmse_cat = np.sqrt(mean_squared_error(y_test, y_pred_cat))

print(f"RMSE de CatBoost (default): {rmse_cat:.2f} euros")

RMSE de CatBoost (default): 1585.09 euros
CPU times: total: 16min 50s
Wall time: 2min 15s
